In [27]:
import os 
import json
import pandas as pd 
import traceback

In [28]:
from langchain.chat_models import ChatOpenAI


In [29]:
from dotenv import load_dotenv
load_dotenv()
KEY=os.getenv('OPENAI_API_KEY')

In [30]:
llm=ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature =0.3)


In [31]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x77d05b81c970>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x77d05b81c3a0>, temperature=0.3, openai_api_key='asuhhskhfkdnvxvcbdfjhvxjhvb', openai_proxy='')

In [32]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [33]:
Response_json={
    '1':{
        'mcq':'multiple choice question',
        'options' : {
             'a':'choice here',
             'b':'choice here',
             'c':'choice here',
             'd':'choice here',
          },
          'correct':'correct answer',
    },
    '2':{
        'mcq':'multiple choice question',
        'options' : {
             'a':'choice here',
             'b':'choice here',
             'c':'choice here',
             'd':'choice here',
          },
          'correct':'correct answer',
    },
    '3':{
        'mcq':'multiple choice question',
        'options' : {
             'a':'choice here',
             'b':'choice here',
             'c':'choice here',
             'd':'choice here',
          },
          'correct':'correct answer',
    },
}

In [34]:
TEMPLATE=""""
Text:{text}
You are expert in Multiple choice questions ,Given the above text,it is your task to \
create  a quiz  of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the  questions are not repeated and check all the  questions to be confirming  the text as well.
Make sure to format your response like RESPONSE_JSON below  use it as a guide . \
Ensure to make {number} MCQ's
##Response json
{response_json}

"""

In [35]:
quiz_generation_prompt=PromptTemplate(
 input_variables=['text','number','subject','tone','respose_json'],
 template=TEMPLATE
)

In [36]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [37]:
TEMPLATE2=""" 
you are an expert  english grammarian and writer .Given a Multiple Choice Quiz for {subject} students,
you need to evaluate the complexity of the question and give  a complete analysis of the quiz .Only use at max 50 words for complexity \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz} 
Check from an expert English Writer of the above quiz:
"""

In [38]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['subject','quiz'],template=TEMPLATE2)

In [39]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key='review',verbose=True )

In [ ]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','respose_json'],output_variables=['quiz','review'],verbose=True)

In [ ]:
file_path='/home/mukesh/Github-my repos/Document-to-MCQ-Generator/data.txt'

In [ ]:
with open(file_path,'r') as file  :
    TEXT=file.read()

In [ ]:
#serialize the python dictionary
json.dumps(Response_json)

In [ ]:
NUMBER=7
SUBJECT='Deep Learning'
TONE='Simple'

In [ ]:
#how to setup token usage tracking in langchain

with get_openai_callback() as cb :
    response=generate_evaluate_chain(
        {
            'text': TEXT,
            'number':NUMBER,
            'subject':SUBJECT,
            'tone': TONE,
            'response_json':json.dumps('response_json')
        }
    )

In [ ]:
print(f'Total Tokens:{cb.total_tokens}')
print(f'Prompt Tokens:{cb.prompt_tokens}')
print(f'Completion Tokens:{cb.completion_tokens}')
print(f'total cost:{cb.total_cost}')